In [1]:
import json
import pandas as pd


"""
TODO:
1. Data sudah bener2 post_rendered, tinggal processing untuk di training BERT
2. Tambah new column untuk ini mungkin namanya `post_processed`
"""

def remove_backslash(text: str) -> str:
    result = text.replace("\\", " ")
    return result

def remove_redundant_white_spaces(text: str) -> str:
    from re import sub
    return sub(r'\s+', ' ', text).strip()


file_path = 'C:/wamp64/www/artofproblemsolving/data/data_post_rendered.csv'
df = pd.read_csv(file_path)


In [17]:
def processing_dollar_sign(text: str):
    """
    Special case LaTex that look like this `\[ ... \]`
    """
    text = text.replace("\[", "$")
    text = text.replace("\]", "$")
    
    def extract_inside_dollar_sign(text: str) -> list:
        """
        Case:
        - "lorem lorem $target 1$ $$target 2 lorem lorem"  -->  ['target 1', '']
        - "lorem lorem $target 1$ $$target 2$$ lorem lorem"  -->  ['target 1', 'target 2']
        """
        import re

        matches: list = re.findall(r'\$+(.*?)\$+', text)
        for i in range(len(matches)):
            matches[i] = matches[i].strip()

        return matches

    
    list_dollar = extract_inside_dollar_sign(text)
    for i in range(len(list_dollar)):
        list_dollar[i] = list_dollar[i].replace("\\", " ")
        list_dollar[i] = list_dollar[i].replace(">", " BIGGERTHANSIGN ")
        list_dollar[i] = list_dollar[i].replace("<", " SMALLERTHANSIGN ")
        list_dollar[i] = list_dollar[i].replace("+", " PLUS_OR_POSITIVESIGN ")
        list_dollar[i] = list_dollar[i].replace("-", " MINUS_OR_NEGATIVESIGN ")
        list_dollar[i] = list_dollar[i].replace("=", " EQUALSIGN ")
        list_dollar[i] = list_dollar[i].replace("!", " FACTORIALSIGN ")
        
    return list_dollar
    

In [18]:
n = 0
test_ = df.iloc[n]['post_canonical']
link_ = df.iloc[n]['link']

print(link_)
print(test_)
print(processing_dollar_sign(test_))


https://artofproblemsolving.com/community/c3381519_2023imo
Determine all composite integers $n>1$ that satisfy the following property: if $d_1, d_2, \ldots, d_k$ are all the positive divisors of $n$ with $1=d_1<d_2<\cdots<d_k=n$, then $d_i$ divides $d_{i+1}+d_{i+2}$ for every $1 \leqslant i \leqslant k-2$
['n biggerthanSIGN 1', 'd_1, d_2,  ldots, d_k', 'n', '1 equalSIGN d_1 smallerthanSIGN d_2 smallerthanSIGN  cdots smallerthanSIGN d_k equalSIGN n', 'd_i', 'd_{i plus_or_positiveSIGN 1} plus_or_positiveSIGN d_{i plus_or_positiveSIGN 2}', '1  leqslant i  leqslant k minus_or_negativeSIGN 2']


In [ ]:
for i in range(df.shape[0]):
    df.iloc[i]['']
    ...

In [25]:
"""
NOTE:
Jangan lupa `$...$ hilangin dolar`
Extrak yang diapit dolar (ada yag `$$...$$`) atau lebih dolarnya

1. Consider `\legslant` and `\leg`
"""

# >
biggerthan = "biggerthan"
# <
smallerthan = "smallerthan"
# +
plus_or_positive = "plus_or_positive"
# -
minus_or_negative = "minus_or_negative"
# / == \frac{}{}

# :

# =
equal = "equal"
# !
factorial = "factorial"

# ^
superscript_variable = "superscript_variable"
# _
subscript_variable = "subscript_variable"
# ^_
combined_supersubscript_variable = "combined_supersubscript_variable"

# ABC --> "uppercase_variable uppercase_variable uppercase_variable"
uppercase_variable = "uppercase_variable"
# n, x, y --> lowercase_variable
lowercase_variable = "lowercase_variable"
